# CPLEX Examples - Teambuilding
###　既知の情報
a). 参加社員は60人で、10部屋（1部屋6人）に割り振る。
b). 新入社員は偶数、既存社員は奇数で、各人を0〜59の数字で表す。
c). 部署に所属する社員は以下の通りである。
 部署        社員番号
  A          0-19 
  B          20-39 
  C          40-44
  D          45-49
  E          50-54
  F          55-59
d). 既存社員が指導できるのは新入社員1人だけである。
e). 部署Aでは、アドバイザー/新入社員のペアは0-1、2-3、4-5、6-7、8-9、10-11 
f). 部署Bでは、アドバイザー/新入社員のペアは20-21、22-23、24-25、26-27、28-29、30-31
g). 部署C,D,E,Fでは、アドバイザー/新入社員は40-41, 42-43, 45-46, 47-48, 50-51, 52-53, 55-56, 57-58

### 制約条件
1. 1つの部屋には、既存社員3人と新入社員3人が入る。
2. 同じ部署から最大4人が参加しなければならない。
3. 部署AとBの社員は同じ部屋には入れない。
4. 部署EとFの社員は同じ部屋には入れない。
5. 新入社員は、その担当アドバイザーと同じ部屋でなくてはならない。
6. 社員5は、社員41または社員51のどちらかと同じ部屋でなくてはならない。
7. 社員15は、社員40か社員51のどちらかと同じ部屋でなくてはならない。
8. 社員25は、社員40か社員50のどちらかと同じ部屋でなくてはならない。
9. 社員20は社員24と同じ部屋か、社員22は社員50と同じ部屋でなくてはならない。

## Step1: ライブラリのダウンロード
最初にdocplexをインストールする必要があります。

In [16]:
import sys
try:
    import docplex.cp
except:
    if hasattr(sys, 'real_prefix'):
        #we are in a virtual env.
        !conda install -y docplex
    else:
        !conda install -y --user docplex

### ライブラリのインポート

In [17]:
from sys import stdout
from collections import namedtuple
from docplex.cp.model import *

## Step2: エンジン（ソルバー）の設定
* Subscribe to our private cloud offer or Decision Optimization on Cloud solve service [here](https://developer.ibm.com/docloud) if you do not want to use a local solver.
* Get the service URL and your personal API key and enter your credentials here if accurate:
* ローカルのソルバーを使用したくない場合は、[こちら](https://developer.ibm.com/docloud) で、弊社のプライベートクラウドオファーまたはDecision Optimization on Cloud solve serviceを購入してください。
* サービスのURLと個人のAPIキーを取得し、正確であればここで認証情報を入力します:

In [18]:
url = None
key = None

## Step3: モデルの設定
### パラメータの設定
モデルのすべてのオブジェクトは、1つのモデルインスタンスに属しています。
まず、問題のパラメータを設定します。
社員は６０人、部屋は１０個です。(a)

In [19]:
nb_persons = 60
persons = range(nb_persons)
print("persons ", list(persons))

nb_rooms = 10
rooms = range(1, nb_rooms+1)
print("rooms ", list(rooms))

persons  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]
rooms  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


社員はA から F までの 6 つの異なる部署に従事しています。

In [20]:
serviceNames = [chr(i) for i in range(65,65+6)]

print("serviceNames ",list(serviceNames))
print(serviceNames.index('A'))

serviceNames  ['A', 'B', 'C', 'D', 'E', 'F']
0


社員には0番から59番までの番号がついてています。新入社員の番号は偶数、既存社員の番号は奇数です。6つの異なる部署と所属する社員の番号を設定します。(b, c)

In [21]:
a = range(0,20) 
b = range(20,40)
c = range(40,45)
d = range(45,50)
e = range(50,55)
f = range(55,60)
service = [set(a), set(b), set(c), set(d), set(e), set(f)]
print("service ",service)
print(service[serviceNames.index('A')])

service  [{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}, {20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39}, {40, 41, 42, 43, 44}, {45, 46, 47, 48, 49}, {50, 51, 52, 53, 54}, {55, 56, 57, 58, 59}]
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}


アドバイザーと新入社員のペア(d)
e). 部署Aでは、アドバイザー/新入社員のペアは0-1、2-3、4-5、6-7、8-9、10-11 
f). 部署Bでは、アドバイザー/新入社員のペアは20-21、22-23、24-25、26-27、28-29、30-31
g). 部署C,D,E,Fでは、アドバイザー/新入社員は40-41, 42-43, 45-46, 47-48, 50-51, 52-53, 55-56, 57-58

In [22]:
pair = namedtuple("pair", ["advisor", "advised"])
pairs = [pair(0, 1), pair(2, 3), pair(4, 5), pair(6, 7), pair(8, 9), pair(10, 11), 
         pair(20, 21), pair(22, 23), pair(24, 25), pair(26, 27), pair(28, 29), pair(30, 31),
         pair(40, 41), pair(42, 43), pair(45, 46), pair(47, 48), pair(50, 51), pair(52, 53), pair(55, 56), pair(57, 58)]
print(pairs)

[pair(advisor=0, advised=1), pair(advisor=2, advised=3), pair(advisor=4, advised=5), pair(advisor=6, advised=7), pair(advisor=8, advised=9), pair(advisor=10, advised=11), pair(advisor=20, advised=21), pair(advisor=22, advised=23), pair(advisor=24, advised=25), pair(advisor=26, advised=27), pair(advisor=28, advised=29), pair(advisor=30, advised=31), pair(advisor=40, advised=41), pair(advisor=42, advised=43), pair(advisor=45, advised=46), pair(advisor=47, advised=48), pair(advisor=50, advised=51), pair(advisor=52, advised=53), pair(advisor=55, advised=56), pair(advisor=57, advised=58)]


### モデルを定義

In [23]:
mdl = CpoModel(name='teambuilding')

### 決定変数を定義。
各社員が所属する部屋の番号をroomという変数名で定義します。

In [24]:
room = mdl.integer_var_list(size=nb_persons, min=list(rooms)[0], max=list(rooms)[-1], name='person')

### 制約条件の設定
1. 1つの部屋には、既存社員3人と新入社員3人が入る。
2. 同じ部署から最大4人が参加しなければならない。

In [25]:
for t in rooms:
    # 1部屋の既存社員は3人（1）
    l_existingemployee = [room[existingemployee] for existingemployee in persons if existingemployee%2==1]
    mdl.add(mdl.count(l_existingemployee, t)==3)
    # 1部屋の新入社員は3人（1）
    l_newemployee = [room[newemployee] for newemployee in persons if newemployee%2==0]
    mdl.add(mdl.count(l_newemployee, t)==3)
    # 同じ部署から最大4人が参加（2）
    for f in range(len(serviceNames)):
        l_person = [room[person] for person in service[f]]
        mdl.add(mdl.count(l_person, t)<=4)

3. 部署AとBの社員は同じ部屋には入れない。
4. 部署EとFの社員は同じ部屋には入れない。
5. 新入社員は、その担当アドバイザーと同じ部屋でなくてはならない。
6. 社員5は、社員41または社員51のどちらかと同じ部屋でなくてはならない。
7. 社員15は、社員40か社員51のどちらかと同じ部屋でなくてはならない。
8. 社員25は、社員40か社員50のどちらかと同じ部屋でなくてはならない。
9. 社員20は社員24と同じ部屋か、社員22は社員50と同じ部屋でなくてはならない。


In [26]:
# 部署AとBの社員は同じ部屋には入れない。(3)
for pA in service[serviceNames.index('A')]:
    for pB in service[serviceNames.index('B')]:
        mdl.add_constraint(room[pA] != room[pB])

# 部署EとFの社員は同じ部屋には入れない。(4)
for pE in service[serviceNames.index('E')]:
    for pF in service[serviceNames.index('F')]:
        mdl.add_constraint(room[pE] != room[pF])

# 新入社員は、その担当アドバイザーと同じ部屋でなくてはならない。(5)
for a in pairs:
    mdl.add(room[a.advisor]==room[a.advised])

# 社員5は、社員41または社員51のどちらかと同じ部屋でなくてはならない。(6)
mdl.add((room[5]==room[41]) | (room[5]==room[51]))

# 社員15は、社員40か社員51のどちらかと同じ部屋でなくてはならない。(7)
mdl.add((room[15]==room[40]) | (room[15]==room[51])) 

# 社員25は、社員40か社員50のどちらかと同じ部屋でなくてはならない。(8)
mdl.add((room[25]==room[40]) | (room[25]==room[50]))  

# 社員20は社員24と同じ部屋か、社員22は社員50と同じ部屋でなくてはならない。(9)
mdl.add((room[20]==room[24]) | (room[22]==room[50]))


モデルの概要を表示

In [27]:
mdl.print_information()

Model: teambuilding
 - source file: <ipython-input-23-7676d5bec454>
 - modeling time: 0.05 sec
 - number of integer variables:  60
 - number of interval variables: 0
 - number of sequence variables: 0
 - number of state functions:    0
 - number of float variables:    0
 - number of constraints:        529
 - number of root expressions:   529
 - number of expression nodes:   845
 - operations:                   count: 80, diff: 425, equal: 48, lessOrEqual: 60, or: 4


問題を解く

In [28]:
print("\nSolving model....")
msol = mdl.solve(agent='local',TimeLimit=10, RandomSeed=1)


Solving model....
 ! --------------------------------------------------- CP Optimizer 20.1.0.0 --
 ! Satisfiability problem - 60 variables, 529 constraints
 ! Presolve      : 80 extractables eliminated, 8 constraints generated
 ! TimeLimit            = 10
 ! RandomSeed           = 1
 ! Initial process time : 0.04s (0.03s extraction + 0.01s propagation)
 !  . Log search space  : 132.9 (before), 132.9 (after)
 !  . Memory usage      : 397.2 kB (before), 397.2 kB (after)
 ! Using parallel search with 8 workers.
 ! ----------------------------------------------------------------------------
 !               Branches  Non-fixed    W       Branch decision
 *                    123  0.06s        1         3  = person_55
 ! ----------------------------------------------------------------------------
 ! Search completed, 1 solution found.
 ! ----------------------------------------------------------------------------
 ! Number of branches     : 1148
 ! Number of fails        : 443
 ! Total mem

In [29]:
print("Solve status: " + msol.get_solve_status())
if msol.is_solution():
    stdout.write("Solve time: " + str(msol.get_solve_time()) + "\n")
else:
    stdout.write("No solution found\n")

Solve status: Feasible
Solve time: 0.0599999


結果の表示

In [30]:
msol.print_solution()

-------------------------------------------------------------------------------
Model constraints: 529, variables: integer: 60, interval: 0, sequence: 0
Solve status: Feasible
Search status: SearchCompleted, stop cause: SearchHasNotBeenStopped
Solve time: 0.06 sec
-------------------------------------------------------------------------------
Variables:
   person_0 = 5
   person_1 = 5
   person_2 = 5
   person_3 = 5
   person_4 = 2
   person_5 = 2
   person_6 = 10
   person_7 = 10
   person_8 = 6
   person_9 = 6
   person_10 = 1
   person_11 = 1
   person_12 = 2
   person_13 = 1
   person_14 = 10
   person_15 = 2
   person_16 = 1
   person_17 = 6
   person_18 = 6
   person_19 = 10
   person_20 = 8
   person_21 = 8
   person_22 = 9
   person_23 = 9
   person_24 = 8
   person_25 = 8
   person_26 = 3
   person_27 = 3
   person_28 = 3
   person_29 = 3
   person_30 = 7
   person_31 = 7
   person_32 = 9
   person_33 = 7
   person_34 = 4
   person_35 = 4
   person_36 = 7
   person_37 = 4
   p